In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#path to drive folder
%cd /content/drive/MyDrive/capstone/FinQ&A/

/content/drive/MyDrive/capstone/FinQ&A


In [3]:
!pip install --q --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import load_from_disk

# Load the preprocessed dataset
berttrain_dataset = load_from_disk('preprocessed_train_dataset')

val_dataset = load_from_disk('preprocessed_val_dataset')
test_dataset = load_from_disk('preprocessed_test_dataset')

In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load pre-trained BERT model and tokenizer
model1 = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
tokenizer1 = AutoTokenizer.from_pretrained("bert-base-uncased")

# Freeze all layers
for param in model1.parameters():
    param.requires_grad = False

# Unfreeze the last 3 encoder layers
for layer in model1.bert.encoder.layer[-3:]:
    for param in layer.parameters():
        param.requires_grad = True


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
import logging
from transformers import Trainer, TrainingArguments
import os

# Create the directory if it doesn't exist
log_dir = '/content/drive/MyDrive/capstone/FinQ&A/results'
os.makedirs(log_dir, exist_ok=True)

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
console_handler = logging.StreamHandler()
logger.addHandler(console_handler)
file_handler = logging.FileHandler(f'{log_dir}/hyperparameter_output.log')
logger.addHandler(file_handler)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    max_grad_norm=1.0,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_steps=100,
    fp16=True,
    no_cuda=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="wandb"  # Enable logging to Weights & Biases
)

# Initialize Trainer
trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=berttrain_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_result = trainer.evaluate(test_dataset)
print("Test Result:", eval_result)
logger.info(f"Test results: {eval_result}")

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,1.989100,1.908991
2,1.696900,1.711688
3,1.740300,1.598614
4,1.392500,1.597467
5,1.406700,1.580574
6,1.141400,1.523438
7,1.013400,1.563379
8,1.075100,1.541460
9,0.966100,1.531371
10,1.079100,1.595997


Test Result: {'eval_loss': 1.522751808166504, 'eval_runtime': 14.2783, 'eval_samples_per_second': 116.821, 'eval_steps_per_second': 29.205, 'epoch': 20.0}


In [6]:
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.2 MB/s eta 0:00:00


In [7]:
!pip install -q optuna-integration[pytorch_lightning]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 46.8 MB/s eta 0:00:00


In [11]:
import optuna
import logging
from transformers import Trainer, TrainingArguments, BertModel
from optuna.integration import PyTorchLightningPruningCallback

def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 20)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./paramter_results/{trial.number}",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=4,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        max_grad_norm=1.0,
        logging_dir=f"./paramter_logs/{trial.number}",
        logging_steps=10,
        save_strategy="epoch",
        save_steps=100,
    )

    # Create Trainer
    trainer = Trainer(
        model=model1,
        args=training_args,
        train_dataset=berttrain_dataset,
        eval_dataset=val_dataset,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()

    return eval_result["eval_loss"]

def run_optuna_study():
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=5)
    return study.best_params

def main():
    # Set up logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    console_handler = logging.StreamHandler()
    logger.addHandler(console_handler)
    file_handler = logging.FileHandler('hyperparameter_output.log')
    logger.addHandler(file_handler)

    try:
        # Run Optuna study to find best hyperparameters
        best_params = run_optuna_study()

        # Use best hyperparameters to train final model
        training_args = TrainingArguments(
            output_dir="./hyperparameter_results_best",
            evaluation_strategy="epoch",
            **best_params,
            per_device_eval_batch_size=4,
            max_grad_norm=1.0,
            logging_dir="./logs_best",
            logging_steps=10,
            save_strategy="epoch",
            save_steps=100,
        )

        trainer = Trainer(
            model=model1,
            args=training_args,
            train_dataset=berttrain_dataset,
            eval_dataset=val_dataset,
        )

        # Train the model with best hyperparameters
        trainer.train()

        # Evaluate on test set
        test_results = trainer.evaluate(test_dataset)
        logger.info(f"Test results: {test_results}")

    except Exception as e:
        logger.exception("An error occurred during execution")

if __name__ == "__main__":
    main()

[I 2024-11-06 08:15:47,688] A new study created in memory with name: no-name-51f101c7-3ee7-4313-a817-a6a483058d9d
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,2.217500,2.083498
2,2.220900,2.155168
3,2.610400,2.300476
4,2.210800,2.278320
5,2.780400,2.489045
6,2.329600,2.282405
7,2.701900,2.699867
8,2.376600,2.409683
9,2.036000,2.090488
10,2.475000,2.038989


[I 2024-11-06 08:55:58,892] Trial 0 finished with value: 1.8937350511550903 and parameters: {'learning_rate': 0.0006863374945099727, 'per_device_train_batch_size': 16, 'num_train_epochs': 15, 'weight_decay': 0.010569636979879353}. Best is trial 0 with value: 1.8937350511550903.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,5.505500,5.479445
2,6.240300,6.238281
3,6.232300,6.238281


[I 2024-11-06 09:04:38,365] Trial 1 finished with value: 6.23828125 and parameters: {'learning_rate': 0.0008318171663210945, 'per_device_train_batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 0.08646315583554604}. Best is trial 0 with value: 1.8937350511550903.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


...

Epoch,Training Loss,Validation Loss
1,4.559300,3.647703
2,4.126300,3.809196
3,3.352300,3.306983
4,3.197500,2.986453
5,2.988200,2.879440
6,3.004000,2.820395
7,2.958600,2.806126
8,2.961400,3.067534
9,3.191100,2.935495
10,3.789100,2.944183


[I 2024-11-06 09:51:27,509] Trial 3 finished with value: 2.944182872772217 and parameters: {'learning_rate': 0.00015820367967282197, 'per_device_train_batch_size': 8, 'num_train_epochs': 10, 'weight_decay': 0.02219837726227063}. Best is trial 0 with value: 1.8937350511550903.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,3.039100,2.773701
2,3.158600,2.884129
3,3.426800,2.806160
4,2.991800,2.810805
5,3.408700,2.799356
6,2.955300,2.712912
7,2.759700,2.748447
8,2.959100,2.796508
9,3.089200,2.957741
10,3.355000,2.992797


[I 2024-11-06 10:25:52,599] Trial 4 finished with value: 3.1880033016204834 and parameters: {'learning_rate': 3.7156839394835826e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 13, 'weight_decay': 0.0840577282758745}. Best is trial 0 with value: 1.8937350511550903.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.000000,6.083899
2,0.000000,6.083899
3,0.000000,6.083899
4,0.000000,6.083899
5,0.000000,6.083899
6,0.000000,6.083899
7,0.000000,6.083899
8,0.000000,6.083899
9,0.000000,6.083899
10,0.000000,6.083899


In [ ]:
import weave
import optuna
import logging
from transformers import Trainer, TrainingArguments, BertModel
from optuna.integration import PyTorchLightningPruningCallback

@weave.op()
def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 20)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./paramter_results/{trial.number}",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=4,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        max_grad_norm=1.0,
        logging_dir=f"./paramter_logs/{trial.number}",
        logging_steps=10,
        save_strategy="epoch",
        save_steps=100,
    )

    # Create Trainer
    trainer = Trainer(
        model=model1,
        args=training_args,
        train_dataset=berttrain_dataset,
        eval_dataset=val_dataset,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()

    return eval_result["eval_loss"]

@weave.op()
def run_optuna_study():
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=5)
    return study.best_params

@weave.op()
def main():

    weave.init('hyperparameter tuning using optuna')
    # Set up logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    console_handler = logging.StreamHandler()
    logger.addHandler(console_handler)
    file_handler = logging.FileHandler('hyperparameter_output.log')
    logger.addHandler(file_handler)

    try:
        # Run Optuna study to find best hyperparameters
        best_params = run_optuna_study()

        # Use best hyperparameters to train final model
        training_args = TrainingArguments(
            output_dir="./hyperparameter_results_best",
            evaluation_strategy="epoch",
            **best_params,
            per_device_eval_batch_size=4,
            max_grad_norm=1.0,
            logging_dir="./logs_best",
            logging_steps=10,
            save_strategy="epoch",
            save_steps=100,
        )

        trainer = weave.Trainer(
            model=model1,
            args=training_args,
            train_dataset=berttrain_dataset,
            eval_dataset=val_dataset,
        )

        # Train the model with best hyperparameters
        trainer.train()

        # Evaluate on test set
        test_results = trainer.evaluate(test_dataset)
        logger.info(f"Test results: {test_results}")
        print(f"Test results: {test_results}")

    except Exception as e:
        logger.exception("An error occurred during execution")

if __name__ == "__main__":
    main()